First, install the fhir.resources library.

In [ ]:
!pip install fhir.resources
#!pip install transformers
!pip install openai

The following script will extract observation resources from a questionnaire response resource.

# Modified script to integrate patient extraction service and the output
Here is how I'd probably include it.

In [2]:
from logging import error
import json
from fhir.resources.observation import Observation
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.patient import Patient
from openai import OpenAI
from google.colab import userdata
# from transformers import pipeline, Conversation

In [3]:
def extract_units(questionnaire_file):
    units_map = {}
    with open(questionnaire_file, 'r') as f:
        questionnaire = json.load(f)
        for item in questionnaire['item']:
            link_id = item['linkId']
            if 'extension' in item:
                for extension in item['extension']:
                    if 'url' in extension and extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaire-unit':
                        units_map[link_id] = extension['valueCoding']['display']
    return units_map

def create_observation(link_id, text, answer, units=None):
    # Set code representing the question
    code = CodeableConcept(
        coding=[Coding(
            system="http://loinc.org",
            code=link_id.lstrip('/'),
            display=text
        )]
    )

    # Set status to 'final' as these are completed observations
    observation = Observation(code=code, status="final")

    # Set value based on the answer type
    if 'valueDecimal' in answer:
        value = float(answer['valueDecimal'])
        if units:  # Check if units are provided
            observation.valueQuantity = {
                "value": value,
                "unit": units
            }
        else:
            observation.valueQuantity = {"value": value}
    elif 'valueString' in answer:
        observation.valueString = answer['valueString']
    elif 'valueCodeableConcept' in answer:
        value_codeable_concept = CodeableConcept(
            coding=[Coding(
                system=answer['valueCodeableConcept']['system'],
                code=answer['valueCodeableConcept']['code'],
                display=answer['valueCodeableConcept']['display']
            )]
        )
        observation.valueCodeableConcept = value_codeable_concept

    return observation

def extract_patient_id(patient_dict):
    if 'id' in patient_dict:
        return patient_dict['id']
    else:
        raise ValueError("Patient Id not found")

from fhir.resources.bundle import Bundle, BundleEntry

def consolidate_resources(patient, observations_list):
    # instantiate a bundle of type transaction
    bundle = Bundle(type = "transaction")

    # Create a bundle entry for the patient
    patient_entry = BundleEntry(
        resource=patient,
        request={
            "method": "POST",
            "url": "Patient"
        }
    )
    bundle.entry = [patient_entry]

    # Create bundle entries for each observation
    for observation in observations_list:
        observation_entry = BundleEntry(
            resource=observation,
            request={
                "method": "POST",
                "url": "Observation"
            }
        )
        bundle.entry.append(observation_entry)

    return bundle

def call_llm(bundle):
    #Initialize OpenAI client
    client = OpenAI(api_key=userdata.get('openai'))

    # add a prompt to the bundle
    prompt = f"Please convert this fhir bundle resource, which contains a patient and 3 observation resources, into a short summary note for a physician: {bundle}"

    try:
        # Make a completion request to GPT-3
        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=500
        )

        # Get the generated text from the response
        return response.choices[0].text

    except Exception as e:
        # Handle exceptions and print an error message
        print(f"Error: {e}")
        return None

def main():
    # Run the patient extraction service - integrate code here
    # Load extracted patient json into a dict
    with open('extracted_patient.json', 'r') as patient_json:
        patient_dict = json.load(patient_json)

    # Print the loaded patient
    print("\n**Here is our loaded patient**")
    print(json.dumps(patient_dict, indent=4))

    # Convert dictionary to Patient resource
    patient = Patient(**patient_dict)

    # Extract id from patient resource
    print("\nRunning extraction function...")
    extracted_patient_id = extract_patient_id(patient_dict)

    # Step 2: Return just the id
    print("\nExtracted patient id:", extracted_patient_id,"\n")

    # Extract units from questionnaire file
    units_map = extract_units('ht_wt_panel-q.json')

    # Parse questionnaire response JSON file
    with open('ht_wt_panel-qr.json', 'r') as f:
        questionnaire_response = json.load(f)

    # instantiate list of observations
    observations_list = []

    # Iterate through each item in the questionnaire response
    for item in questionnaire_response['item']:
        link_id = item['linkId']
        text = item['text']
        answer = item.get('answer', [{}])[0]

        # Get units for the current question
        units = units_map.get(link_id)

        # Create Observation resource for the item
        observation = create_observation(link_id, text, answer, units)
        observations_list.append(observation)
        # you would modify this to now pass the extracted_patient_id like this
        # observation = create_observation(link_id, text, answer, units, extracted_patient_id)

        # Print the observation
        print(observation.json(indent=4))

    # Consolidate the patient resource and the observation resources into a single bundle
    bundle = consolidate_resources(patient, observations_list)

    # Print the bundle
    print("\n**Here is the consolidated FHIR bundle**")
    print(bundle.json(indent=4))

    # Use ChatGPT to convert bundle to a clinical note
    response = call_llm(bundle)

    # print response
    print(response)

if __name__ == "__main__":
    main()



**Here is our loaded patient**
{
    "resourceType": "Patient",
    "id": "8247202",
    "name": [
        {
            "use": "official",
            "family": "Simpson",
            "given": [
                "Bart"
            ]
        }
    ],
    "gender": "male",
    "birthDate": "2000-06-05"
}

Running extraction function...

Extracted patient id: 8247202 

{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "9732277310580",
                "display": "What brings you to the doctor today?"
            }
        ]
    },
    "valueString": "I have pain in my stomach"
}
{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "29463-7",
                "display": "What is your weight?"
            }
        ]
    },
    "valueQuantity"